In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch
import numpy as np
from pow.compute.compute import Compute
from pow.models.utils import Params


In [ ]:
from pow.compute.gpu_group import create_gpu_groups
create_gpu_groups()

In [ ]:
# params = Params(
#     dim=1024,
#     n_layers=64,
#     n_heads=32,
#     n_kv_heads=32,
#     vocab_size=8196,
#     ffn_dim_multiplier=10.0,
#     multiple_of=4*2048,
#     norm_eps=1e-5,
#     rope_theta=10000.0,
#     use_scaled_rope=False,
#     seq_len=128,
# )
params = Params(
    dim=1024,
    n_layers=32,
    n_heads=32,
    n_kv_heads=32,
    vocab_size=8196, 
    ffn_dim_multiplier=10.0,
    multiple_of=2048,
    norm_eps=1e-05,
    rope_theta=10000.0,
    use_scaled_rope=False,
    seq_len=128
)


In [ ]:
block_hash = "test_block_hash_debug_12345"
block_height = 12345
public_key = "debug_public_key_67890"
r_target = 0.5
devices = ["cuda:0"]

compute = Compute(
    params=params,
    block_hash=block_hash,
    block_height=block_height,
    public_key=public_key,
    r_target=r_target,
    devices=devices,
    node_id=0,
)


In [ ]:
test_nonces = list(range(400))

future_result = compute(
    nonces=test_nonces,
    public_key=public_key,
    target=compute.target,
    next_nonces=None,
    use_cache=False
)

proof_batch = future_result.result()

In [ ]:
from pow.compute.gpu_group import create_gpu_groups
create_gpu_groups()

In [ ]:
compute.model.module.freqs_cis

In [ ]:
proof_batch

In [ ]:
compute.model.module

In [ ]:
proof_batch.dist = proof_batch.dist.tolist()

In [ ]:
import json

with open("proof_batch.json", "w") as f:
    json.dump(proof_batch.__dict__, f)


In [ ]:
import json
from pow.data import ProofBatch

with open("proof_batch.json", "r") as f:
    proof_batch1 = json.load(f)

proof_batch1 = ProofBatch(**proof_batch1)

print(proof_batch1)

In [ ]:
validated_batch = compute.validate(proof_batch1)
print(f"Original distances: {proof_batch1.dist}")
print(f"Validated distances: {validated_batch.dist}")

distance_differences = [abs(d1 - d2) for d1, d2 in zip(proof_batch1.dist, validated_batch.dist)]
max_diff = max(distance_differences)
print(f"Maximum difference between original and validated: {max_diff}")
print(f"Validation {'✅ PASSED' if max_diff < 1e-6 else '❌ FAILED'}")
